In [ ]:
# Imports
import pandas as pd
import numpy as np

df_players = pd.read_csv('dataset/processed/players_teams_processed.csv')
df_teams = pd.read_csv('dataset/processed/teams_processed.csv')
df_awards = pd.read_csv('dataset/processed/awards_processed.csv')

league_df_stats = pd.DataFrame()
league_df_stats['lg_AST'] = df_teams.groupby('year')['o_asts'].sum().reset_index(drop=True)
league_df_stats['lg_FG'] = df_teams.groupby('year')['o_fgm'].sum().reset_index(drop=True)
league_df_stats['lg_FT'] = df_teams.groupby('year')['o_ftm'].sum().reset_index(drop=True)
league_df_stats['lg_PTS'] = df_teams.groupby('year')['o_pts'].sum().reset_index(drop=True)
league_df_stats['lg_FGA'] = df_teams.groupby('year')['o_fga'].sum().reset_index(drop=True)
league_df_stats['lg_ORB'] = df_teams.groupby('year')['o_oreb'].sum().reset_index(drop=True)
league_df_stats['lg_TOV'] = df_teams.groupby('year')['o_to'].sum().reset_index(drop=True) 
league_df_stats['lg_FTA'] = df_teams.groupby('year')['o_fta'].sum().reset_index(drop=True) 
league_df_stats['lg_TRB'] = df_teams.groupby('year')['o_reb'].sum().reset_index(drop=True) 
league_df_stats['lg_PF'] = df_teams.groupby('year')['o_pf'].sum().reset_index(drop=True) 

stats_df_players = pd.merge(df_players, df_teams, on=['tmID', 'year'], how='left')

stats_df_players['factor'] = (2/3) - (0.5* (league_df_stats['lg_AST'] / league_df_stats['lg_FG'])) / (2 * (league_df_stats['lg_FG'] / league_df_stats['lg_FT']))
stats_df_players['vop'] = league_df_stats['lg_PTS'] / (league_df_stats['lg_FGA'] - league_df_stats['lg_ORB'] + league_df_stats['lg_TOV'] + 0.44 * league_df_stats['lg_FTA'])
stats_df_players['drb'] = (league_df_stats['lg_TRB'] - league_df_stats['lg_ORB']) / league_df_stats['lg_TRB']
#stats_df_players['PER'] = (1/ (df_players['minutes'] + df_players['PostMinutes'])) * ((df_players['threeMade'] + df_players['PostthreeMade']) + 2/3 * (df_players['assists']+ df_players['PostAssists']) + (2 - stats_df_players['factor'] *  (stats_df_players['o_asts'] / stats_df_players['o_fgm'])) * (stats_df_players['fgMade'] + stats_df_players['PostfgMade']) + ((stats_df_players['ftMade'] + stats_df_players['PostftMade']) * 0.5 * (1 + (1 - (stats_df_players['o_asts'] / stats_df_players['o_fgm'])) + (2/3) * (stats_df_players['o_asts'] / stats_df_players['o_fgm']))) - stats_df_players['vop'] * (stats_df_players['turnovers'] + stats_df_players['PostTurnovers']) - stats_df_players['vop'] * stats_df_players['drb'] * ((stats_df_players['fgAttempted'] + stats_df_players['PostfgAttempted']) - (stats_df_players['fgMade'] + stats_df_players['PostfgMade'])) - stats_df_players['vop'])

stats_df_players['PER'] = (
    (1 / (df_players['minutes'] + df_players['PostMinutes'])).replace([np.inf, -np.inf, np.nan], 0) * 
    ((df_players['threeMade'] + df_players['PostthreeMade'])
      +  (2 / 3) * (df_players['assists'] + df_players['PostAssists'])
      +  (2 - stats_df_players['factor'] * (stats_df_players['o_asts'] / stats_df_players['o_fgm']).replace([np.inf, -np.inf, np.nan], 0)) * (stats_df_players['fgMade'] + stats_df_players['PostfgMade']) 
      +  ((stats_df_players['ftMade'] + stats_df_players['PostftMade']) * 0.5 * (1 + (1 - (stats_df_players['o_asts'] / stats_df_players['o_fgm']).replace([np.inf, -np.inf, np.nan], 0)) + (2 / 3) * (stats_df_players['o_asts'] / stats_df_players['o_fgm']).replace([np.inf, -np.inf, np.nan], 0))) 
      - stats_df_players['vop'] * (stats_df_players['turnovers'] + stats_df_players['PostTurnovers']) 
      - stats_df_players['vop'] * stats_df_players['drb'] * ((stats_df_players['fgAttempted'] + stats_df_players['PostfgAttempted']) - (stats_df_players['fgMade'] + stats_df_players['PostfgMade'])) 
      - stats_df_players['vop'] * 0.44 * (0.44 + (0.56 * stats_df_players['drb'])) * ((stats_df_players['ftAttempted'] + stats_df_players['PostftAttempted']) - (stats_df_players['ftMade'] + stats_df_players['PostftMade']))
      + stats_df_players['vop'] * (1 - stats_df_players['drb']) * (stats_df_players['rebounds'] - stats_df_players['oRebounds'])
      + stats_df_players['vop'] * stats_df_players['drb'] * stats_df_players['oRebounds']
      + stats_df_players['vop'] * (stats_df_players['steals'] * stats_df_players['PostSteals'])
      + stats_df_players['vop'] * stats_df_players['drb'] * (stats_df_players['blocks'] * stats_df_players['PostBlocks'])
      - (stats_df_players['PF'] + stats_df_players['PostPF']) * ((league_df_stats['lg_FT'] / league_df_stats['lg_PF']) - 0.44 * (league_df_stats['lg_FTA'] / league_df_stats['lg_PF']) * stats_df_players['vop'])
    )
)

stats_df_players = stats_df_players[['playerID', 'year', 'tmID', 'PER']].copy()
sorted_stats_df = stats_df_players.sort_values(by='PER', ascending=False)
print(sorted_stats_df)
# Display the sorted DataFrame



        playerID  year  stint tmID  GP_x  GS  minutes  points  oRebounds  \
2     abrossv01w     4      0  MIN    30  25      792     318         44   
7     abrossv01w     9      0  CON     6   0      107      34          3   
6     abrossv01w     8      0  MIN    34  29      843     345         53   
0     abrossv01w     2      0  MIN    26  23      846     343         43   
5     abrossv01w     7      0  MIN    34   2      724     263         44   
...          ...   ...    ...  ...   ...  ..      ...     ...        ...   
1871  zakalok01w     3      2  PHO     5   0       37       6          0   
1872   zarafr01w     6      0  SEA    34   4      413      90         11   
1873  zellosh01w    10      0  DET    34   4      802     406         25   
1874  zirkozu01w     4      0  WAS     6   0       30      11          0   
1875   zollsh01w     9      0  MIN     6   0       30      10          1   

      dRebounds  ...  homeL  awayW  awayL  confW  confL   min    factor  \
2           